In [32]:
from pathlib import Path
import shutil
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
cwd = os.getcwd()

In [33]:
# number of samples per class to select
N = 750

# train val split
SPLIT = 2/3

# size for cutting out of frames
CUT_SIZE = 400

# size of frames to be fed to the network
SIZE = 256

In [34]:
p = Path(cwd, 'raw_data')

### get frames of video

In [35]:
# p = p / 'vid001.mp4'
# os.system("ffmpeg -i "+str(p)+" -r 1/1 raw_data/frames/%05d.bmp")

### get data frame with monkey bb's

drop all monky-less frames

---

results file contents
- rows of: '{frame},{id},{x1},{y1},{w},{h},{score},{cls_id},1\n'
  - class ID's: monkey,patch,kong,branch,XBI
  - frame ID's are all +1

In [36]:
bb_df = pd.read_csv(str((p / 'results.txt')), sep=',', names=['frame','id','x1','y1','w','h','score','cls_id','1'])

# drop last column thats always 1 and score as we ingnore it either way
bb_df.drop(['1', 'score'], axis=1, inplace=True)

# drop all rows not containing a bb for a monkey
bb_df = bb_df[bb_df.cls_id == 0]

# drop column containing cls_id
bb_df.drop('cls_id', axis=1, inplace=True)

# substract 1 from frame column
bb_df.frame -= 1

# drop id column as we do not have a need for it
bb_df.drop('id', axis=1, inplace=True)

# set frame column as index column
bb_df.set_index('frame', inplace=True)

bb_df

,x1,y1,w,h
frame,,,,
0,420.844788,186.737427,224.581787,157.413940
1,421.616690,187.334521,223.154584,156.398481
2,421.174133,187.133186,223.494356,156.709724
3,421.281979,187.206122,223.030090,156.409062
4,421.140797,187.421083,221.930143,155.795984
...,...,...,...,...
20995,620.534945,307.258562,195.949614,179.308152
20996,619.245913,308.421343,192.397138,177.108591
20997,618.442531,308.309059,191.720266,177.593681


### get data frame with pose information

---

- drop rows containing interactions
- get set of distinct poses

In [37]:
label_df = pd.read_csv(str((p / 'hum_2021-09-21_M1-2_cam19415032_vid001.mp4.txt')), sep=' ', names=['frame','from','to','pose_or_action'])

# drop all rows containing interactions
label_df = label_df[label_df.to == '-']

# drop to column as it is no longer needed
label_df.drop('to', axis=1, inplace=True)

# rename last column as contents are now cleaned up
label_df.rename(columns={'pose_or_action': 'pose'}, inplace=True)

# can drop from column there is only one monkey either way...even if it is detected with multiple ID's
label_df.drop('from', axis=1, inplace=True)

# set frame column as index column
label_df.set_index('frame', inplace=True)

label_df

,pose
frame,
0,walking
1,walking
2,walking
3,walking
4,walking
...,...
20986,walking
20987,walking
20988,walking


In [38]:
labels = label_df.pose.unique()
labels

array(['walking', 'standing4legs', 'sitting', 'standing2legs'],
      dtype=object)

### create combined df

---

combine df's of monkey bb's and according pose label

join tables on frame number
- use merge instead of join (takes by default inner join and thus only matches intersection of keys)




In [39]:
joined_df = bb_df.merge(label_df, left_index=True, right_index=True)

# turn bb values to int
joined_df[['x1','y1','w','h']] = joined_df[['x1','y1','w','h']].astype(int)

joined_df

,x1,y1,w,h,pose
frame,,,,,
0,420,186,224,157,walking
1,421,187,223,156,walking
2,421,187,223,156,walking
3,421,187,223,156,walking
4,421,187,221,155,walking
...,...,...,...,...,...
20986,621,302,204,175,walking
20987,622,303,202,176,walking
20988,621,303,204,179,walking


### generate trainings data

---

- get similar numbers of different poses in train and validation sets
- *future* also get similar numbers accross different videos

In [40]:
# get sample count per label
for l in labels:
    print(l, len(joined_df[joined_df.pose == l]))

walking 1339
standing4legs 1709
sitting 15385
standing2legs 2123


In [41]:
# sample N elements per pose
samples = {l: joined_df[joined_df.pose==l].sample(N).drop('pose', axis=1)
                        for l in labels
                    }
samples

{'walking':         x1   y1    w    h
 frame                    
 733    293  121  145  186
 112    282  140  226  189
 12869  364  197  103  215
 12864  357  191  108  220
 20967  634  289  189  152
 ...    ...  ...  ...  ...
 12961  348  241  136  219
 18643  428  422  268  181
 86     331  159  203  169
 20038  672  349  162  214
 20563  722  334  173  154
 
 [750 rows x 4 columns],
 'standing4legs':         x1   y1    w    h
 frame                    
 19654  636  342  185  229
 19772  641  332  194  237
 688    301  137  148  174
 20744  625  234  206  192
 19680  639  333  198  236
 ...    ...  ...  ...  ...
 20734  624  233  210  196
 700    293  130  155  178
 20833  635  235  198  190
 377    293  120  154  189
 20403  758  332  202  180
 
 [750 rows x 4 columns],
 'sitting':         x1   y1    w    h
 frame                    
 10906  297   99  116  169
 5530   301  100  107  167
 2542   298   92  105  174
 1053   263  104  123  161
 2300   302   91  105  176
 ...    ...  ...

In [42]:
# # create train and val split
samples_train = {l: samples[l].iloc[:int(SPLIT*N)]
                        for l in labels
                    }
samples_validation = {l: samples[l].iloc[int(SPLIT*N):]
                        for l in labels
                    }
# from sklearn.model_selection import train_test_split
# X_train, X_validation = train_test_split(samples['walking'],  train_size=SPLIT)
# X_train, X_validation

### collect images and save at correct location

---

- read in images using opencv
- cut images to 256x256 with bb in center
- save images in train / validation folder
  - using subfolders per label

In [43]:
cap = cv2.VideoCapture(str((p / 'vid001.mp4')))
samples[labels[0]].iloc[0]


x1    293
y1    121
w     145
h     186
Name: 733, dtype: int64

In [51]:
for l in labels:
    print(l)
    for index, row in samples_validation[l].iterrows():
        x1, y1, w, h = row
        # print(index, row)
        cap.set(cv2.CAP_PROP_POS_FRAMES, index)
        ret, frame = cap.read()
        # print(frame.shape)
        (fh, fw, c) = frame.shape
        
        # if w <= CUT_SIZE and h <= CUT_SIZE:
        # corners = [[x1 - (SIZE-w)//2, y1 - (SIZE-h)//2],
        #             [x1 + w + (SIZE-w)//2, y1 - (SIZE-h)//2],
        #             [x1 + w + (SIZE-w)//2, y1 + h + (SIZE-h)//2],
        #             [x1 - (SIZE-w)//2, y1 + h + (SIZE-h)//2]
        #         ]
        y = np.clip([y1 - (CUT_SIZE-h)//2, y1 + h + (CUT_SIZE-h)//2], 0, fh)
        x = np.clip([x1 - (CUT_SIZE-w)//2, x1 + w + (CUT_SIZE-w)//2], 0, fw)
        frame = frame[y[0]:y[1],x[0]:x[1],:]

        # pad: top, bottom, left, right
        w_pad, h_pad = max(0, w - CUT_SIZE), max(0, h - CUT_SIZE)
        if w_pad > 0 or h_pad > 0:
            print(h_pad//2, h_pad//2+h_pad%2)
            frame = np.pad(frame, ((h_pad//2, h_pad//2+h_pad%2), (w_pad//2, w_pad//2+w_pad%2)), 'constant')

        # padded_frame = cv2.copyMakeBorder(frame, frame.shape[])
        # else:
        #     # this is a problem!!!!
        #     pass
        print(index)
        cv2.imwrite(str((p / 'poses' / 'validation_data' / l))+"/{:05d}.png".format(index), frame)

        

        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # plt.imshow(frame)
        # plt.title(l)
        # plt.show()
        # print(x1, y1, w, h)
        # break
   

walking
20341
12847
12935
13274
12841
20141
18674
20079
20652
20094
123
20956
20581
68
12826
18639
12786
20978
20657
12954
12695
20649
12871
12817
20961
190
146
734
12990
20065
20169
20350
20620
13306
119
789
156
93
19999
751
27
13327
13288
12948
36
18642
12757
12791
19984
12889
13350
13345
20621
12946
722
12778
13016
122
20058
783
0
20989
104
12918
720
12907
12719
18661
13256
12726
18706
766
20215
20570
128
12750
20220
20351
20987
20148
796
179
20313
20587
758
132
92
13287
20677
12964
13344
12963
12754
18632
20615
140
13004
20000
20582
55
19991
20678
13310
776
13313
20655
13332
20632
20204
61
12747
20029
20526
20176
13357
20110
20136
12905
18620
755
12730
12725
12913
799
13285
12943
178
12769
20216
18679
18614
20654
12797
20345
12861
20123
12748
13311
20572
20593
82
20358
20984
12844
18626
20613
20953
13
20976
19988
20127
12768
20152
785
18703
12849
20213
12986
12762
13299
20109
12838
12758
12926
20568
109
18678
78
164
12941
20639
17
20538
25
20329
148
12834
81
20536
20026
791
12772
2